In [2]:
import os
os.environ['J_NOTEBOOK'] = '1'

from preprocess.preprocess import *
import PARAMETERS
import sys
sys.path.append('./preprocess/')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from confusion_matrix_pretty_print import print_confusion_matrix
from pathlib import Path

%matplotlib inline


PARAMETERS.PREPROCESS_OPTIMIZATION = True

PARAMETERS.LBP_METHOD = 'riu'
PARAMETERS.METHOD = 'get_pyramid_dataset'
PARAMETERS.INTERPOLATION_ALGORITHM = 'lanczos'
PARAMETERS.BALANCE = False
PARAMETERS.N_SCALES = 5
PARAMETERS.GRAY_INTENSITY = True
PARAMETERS.X2SCALE = True
PARAMETERS.FILE_EXTENSION = PARAMETERS.update_file_extension(PARAMETERS)
PARAMETERS.PLOT = False
PARAMETERS.DATASET = 'DRIVE'

pd.DataFrame({k: getattr(PARAMETERS, k) for k in VALID_PARAMETERS.keys()}, index=['Parameters']).head()

,LBP_METHOD,METHOD,INTERPOLATION_ALGORITHM,BALANCE,N_SCALES,GRAY_INTENSITY,X2SCALE
Parameters,riu,get_pyramid_dataset,lanczos,False,5,True,True


In [3]:
# Get CV folds

import pandas as pd
import numpy as np

preprocess = Preprocess(height=608, width=576)
masks_path = f'../dataset/{PARAMETERS.DATASET}/training/mask/'
masks = sorted(os.listdir(masks_path))[:14]

n_pixels = {}
for i, mask_path in enumerate(masks):
    mask = preprocess.read_img(masks_path + mask_path)
    n_pixels[i] = np.sum(mask > 100)
    
pixel_ref = {}

for k, v in n_pixels.items():
    upper = np.sum(np.array(list(n_pixels.values()))[:k+1])
    pixel_ref[k] = (upper - n_pixels[k], upper)

cv_folds = {}

for i, j in enumerate(range(0, 14, 3)):
    cv_folds[i] = (pixel_ref[j][0], pixel_ref[min(j+2, 13)][1])

In [4]:
import pandas as pd

df_train = pd.DataFrame()
df_test = pd.DataFrame()

#for conf in ['_blue', '_blue_lbp_gb', '_green', '_green_lbp_gb']:
for conf in ['_blue_lbp_gb', '_green_lbp_gb']:

    df_train_riu = pd.read_pickle(f'../DB/DRIVE/extra_features/preprocess_optimization{conf}/train_train_riu_get_pyramid_dataset_lanczos_balance-False_scales-5_x2-True_gray-intensity-True.pkl', compression='gzip')
    df_test_riu = pd.read_pickle(f'../DB/DRIVE/extra_features/preprocess_optimization{conf}/train_test_riu_get_pyramid_dataset_lanczos_balance-False_scales-5_x2-True_gray-intensity-True.pkl', compression='gzip')

    df_train_riu.columns = [c + '_riu' if c not in ['Original', 'label'] else c for c in df_train_riu.columns]
    df_test_riu.columns = [c + '_riu' if c not in ['Original', 'label'] else c for c in df_test_riu.columns]

    if df_train.shape[0] == 0:
        y_train = df_train_riu['label']
        y_test = df_test_riu['label']
    df_train_riu = df_train_riu.drop(['label'], axis=1)
    df_test_riu = df_test_riu.drop(['label'], axis=1)


    df_train_var = pd.read_pickle(f'../DB/DRIVE/extra_features/preprocess_optimization{conf}/train_train_var_get_pyramid_dataset_lanczos_balance-False_scales-5_x2-True_gray-intensity-True.pkl', compression='gzip')
    df_test_var = pd.read_pickle(f'../DB/DRIVE/extra_features/preprocess_optimization{conf}/train_test_var_get_pyramid_dataset_lanczos_balance-False_scales-5_x2-True_gray-intensity-True.pkl', compression='gzip')

    df_train_var.columns = [c + '_var' if c not in ['Original', 'label'] else c for c in df_train_var.columns]
    df_test_var.columns = [c + '_var' if c not in ['Original', 'label'] else c for c in df_test_var.columns]

    df_train_var = df_train_var.drop(['Original', 'label'], axis=1)
    df_test_var = df_test_var.drop(['Original', 'label'], axis=1)

    df_train_temp = pd.concat([df_train_riu, df_train_var], axis=1)
    df_test_temp = pd.concat([df_test_riu, df_test_var], axis=1)
    df_train_temp.columns = [c + conf for c in df_train_temp.columns]
    df_test_temp.columns = [c + conf for c in df_test_temp.columns]

    if df_train.shape[0] > 0:
        df_train_temp.index = df_train.index
        df_test_temp.index = df_test.index
    df_train = pd.concat([df_train, df_train_temp], axis=1)
    df_test = pd.concat([df_test, df_test_temp], axis=1)
    
display(df_train)
display(df_test)

,Original_blue_lbp_gb,1:1_riu_blue_lbp_gb,1:2_riu_blue_lbp_gb,1:4_riu_blue_lbp_gb,1:8_riu_blue_lbp_gb,2:1_1_riu_blue_lbp_gb,2:1_2_riu_blue_lbp_gb,2:1_3_riu_blue_lbp_gb,2:1_4_riu_blue_lbp_gb,1:1_var_blue_lbp_gb,...,2:1_3_riu_green_lbp_gb,2:1_4_riu_green_lbp_gb,1:1_var_green_lbp_gb,1:2_var_green_lbp_gb,1:4_var_green_lbp_gb,1:8_var_green_lbp_gb,2:1_1_var_green_lbp_gb,2:1_2_var_green_lbp_gb,2:1_3_var_green_lbp_gb,2:1_4_var_green_lbp_gb
0,3.0,34.0,8.0,8.0,8.0,1.0,0.0,27.0,18.0,17.008752,...,27.0,27.0,34.970179,2366.455049,2452.908099,1774.988954,3.0,3.0,6.0,4.0
1,3.0,1.0,8.0,8.0,8.0,1.0,1.0,27.0,34.0,49.574616,...,35.0,35.0,54.589251,2366.455049,2452.908099,1774.988954,3.0,4.0,3.0,3.0
2,2.0,35.0,8.0,8.0,8.0,9.0,1.0,27.0,34.0,28.109620,...,35.0,35.0,73.904109,2390.571498,2396.025127,1774.988954,3.0,3.0,4.0,3.0
3,3.0,4.0,8.0,8.0,8.0,1.0,1.0,34.0,34.0,102.533453,...,34.0,35.0,87.514134,2390.571498,2396.025127,1774.988954,1.0,1.0,7.0,13.0
4,3.0,16.0,16.0,8.0,8.0,1.0,2.0,33.0,27.0,74.235429,...,35.0,8.0,87.679605,2372.122827,2396.025127,1774.988954,0.0,0.0,17.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226537,2.0,34.0,34.0,35.0,8.0,16.0,16.0,16.0,16.0,4.784987,...,16.0,16.0,339.349974,411.913183,799.976832,1179.119760,105.0,34.0,54.0,54.0
226538,2.0,35.0,34.0,35.0,8.0,16.0,8.0,16.0,4.0,4.669947,...,16.0,27.0,216.946197,411.913183,799.976832,1179.119760,254.0,3.0,54.0,45.0
226539,2.0,35.0,34.0,34.0,8.0,27.0,34.0,16.0,16.0,4.669947,...,8.0,16.0,159.821459,423.340405,769.846024,1179.119760,173.0,104.0,15.0,4.0
226540,2.0,34.0,34.0,34.0,8.0,35.0,34.0,16.0,16.0,4.784987,...,16.0,24.0,110.285592,423.340405,769.846024,1179.119760,35.0,5.0,2.0,0.0


,Original_blue_lbp_gb,1:1_riu_blue_lbp_gb,1:2_riu_blue_lbp_gb,1:4_riu_blue_lbp_gb,1:8_riu_blue_lbp_gb,2:1_1_riu_blue_lbp_gb,2:1_2_riu_blue_lbp_gb,2:1_3_riu_blue_lbp_gb,2:1_4_riu_blue_lbp_gb,1:1_var_blue_lbp_gb,...,2:1_3_riu_green_lbp_gb,2:1_4_riu_green_lbp_gb,1:1_var_green_lbp_gb,1:2_var_green_lbp_gb,1:4_var_green_lbp_gb,1:8_var_green_lbp_gb,2:1_1_var_green_lbp_gb,2:1_2_var_green_lbp_gb,2:1_3_var_green_lbp_gb,2:1_4_var_green_lbp_gb
0,3.0,27.0,35.0,8.0,8.0,1.0,9.0,34.0,34.0,8.540576,...,35.0,34.0,34.398997,315.689010,1801.462839,1568.031927,2.0,5.0,4.0,3.0
1,3.0,34.0,35.0,8.0,8.0,1.0,9.0,34.0,35.0,4.011545,...,35.0,35.0,24.427897,315.689010,1801.462839,1568.031927,5.0,5.0,3.0,4.0
2,3.0,35.0,18.0,8.0,8.0,9.0,1.0,35.0,34.0,3.715399,...,34.0,34.0,7.085000,315.030495,1801.462839,1568.031927,4.0,2.0,3.0,3.0
3,3.0,35.0,18.0,8.0,8.0,9.0,1.0,34.0,35.0,3.970374,...,35.0,35.0,1.271459,315.030495,1801.462839,1568.031927,4.0,6.0,2.0,1.0
4,3.0,35.0,34.0,8.0,8.0,1.0,1.0,34.0,16.0,2.839092,...,16.0,27.0,6.128713,431.363135,2207.566737,1568.031927,7.0,6.0,16.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226969,4.0,4.0,8.0,8.0,8.0,8.0,8.0,34.0,35.0,36.037325,...,27.0,35.0,40.986932,497.701176,497.373815,428.512230,18.0,28.0,1.0,2.0
226970,3.0,16.0,8.0,8.0,8.0,8.0,27.0,35.0,32.0,31.176642,...,34.0,35.0,41.127693,497.701176,497.373815,428.512230,35.0,32.0,2.0,1.0
226971,3.0,4.0,4.0,8.0,4.0,4.0,34.0,8.0,1.0,33.012700,...,35.0,35.0,63.773086,479.650461,566.545021,485.067545,17.0,13.0,6.0,2.0
226972,3.0,8.0,4.0,8.0,4.0,34.0,27.0,1.0,8.0,35.603141,...,27.0,1.0,21.587353,479.650461,566.545021,485.067545,6.0,3.0,1.0,1.0


In [5]:
# CV

from main import init_clf_and_fit
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

def cross_validate_features(features, verbose=True):
    f1_list = []
    acc_list = []
    if isinstance(features['y_train'], pd.Series):
        features['y_train'] = features['y_train'].to_frame()
    for k, v in cv_folds.items():
        df_train_temp = pd.concat(
            [features['x_train'].iloc[cv_folds[0][0]:v[0], :],
             features['x_train'].iloc[v[1]:cv_folds[4][1], :]], axis=0)
        y_train_temp = pd.concat(
            [features['y_train'].iloc[cv_folds[0][0]:v[0], :],
             features['y_train'].iloc[v[1]:cv_folds[4][1], :]], axis=0)
        clf = init_clf_and_fit(df_train_temp, y_train_temp.values.ravel(), lgb='Num')
        y_predicted = clf.predict(features['x_train'].iloc[v[0]:v[1], :])
        acc = accuracy_score(features['y_train'].iloc[v[0]:v[1], :], y_predicted)
        acc_list.append(acc)
        f1 = f1_score(features['y_train'].iloc[v[0]:v[1], :], y_predicted)
        f1_list.append(f1)
        mat = confusion_matrix(features['y_train'].iloc[v[0]:v[1], :], y_predicted).ravel()
        if verbose:
            print('Accuracy score: ' + str(acc) + '\n')
            print('F1 score: ' + str(f1) + '\n')
            print('Confusion matrix:\n')
            print_confusion_matrix(features['y_train'].iloc[v[0]:v[1], :].values.ravel(), y_predicted.ravel())
            print(f'Sensivity: {int(mat[3]) / (int(mat[3]) + int(mat[2]))}')
            print(f'Specificity: {int(mat[0]) / (int(mat[0]) + int(mat[1]))}')
            print('-------------------------------------')

    return {'acc': np.mean(acc_list), 'f1': np.mean(f1_list)}


cross_validate_features({'x_train': df_train, 'y_train':y_train})

Accuracy score: 0.9272678448008945

F1 score: 0.6743579467435795

Confusion matrix:

     t/p    0.0   1.0 
      0.0 580587 24722 
      1.0 24843 51321 
Sensivity: 0.6738222782416889
Specificity: 0.9591580498555283
-------------------------------------
Accuracy score: 0.9160295804051869

F1 score: 0.6340635191153198

Confusion matrix:

     t/p    0.0   1.0 
      0.0 573583  9222 
      1.0 47893 49482 
Sensivity: 0.5081591784338896
Specificity: 0.9841765255960399
-------------------------------------
Accuracy score: 0.9331493914679498

F1 score: 0.7156364724287736

Confusion matrix:

     t/p    0.0   1.0 
      0.0 579431 14013 
      1.0 31610 57408 
Sensivity: 0.6449032779887214
Specificity: 0.9763869884942808
-------------------------------------
Accuracy score: 0.9365778109850802

F1 score: 0.6656742570495973

Confusion matrix:

     t/p    0.0   1.0 
      0.0 593854 13320 
      1.0 29801 42929 
Sensivity: 0.5902516155644163
Specificity: 0.9780623017454634
------------------

{'acc': 0.9242990348916885, 'f1': 0.667327033167218}

In [6]:
import pickle
from os import listdir

PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/images'
MASK_PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/mask'
LABELS_PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/1st_manual'

def load_images():
    paths = [f"{PATH}/{path}" for path in sorted(listdir(PATH))][:14]
    return [np.asarray(Image.open(path).convert('RGB'))[:, :, 1] for path in paths]

def load_masks():
    paths = [f"{MASK_PATH}/{path}" for path in sorted(listdir(MASK_PATH))][:14]
    return [np.asarray(Image.open(path).convert('L')) > 100 for path in paths]

IMAGES = load_images()
MASKS = load_masks()

with open(r'/home/fer/Drive/Estudios/Master-IA/TFM/lbp-pyramid/convolution/outputs/16544931165874937_200_iterations_corrected/population.pkl', 'rb') as f:
    kernels = pickle.load(f)
    
population, fitness = kernels['population'], kernels['fitness']
individual = population[np.argmax(fitness), :]
n_kernels = 6
k_size = (3, 5, 7)

features = [pd.DataFrame()]*14
count = 0
for j, ks in enumerate(k_size):
    k_len = int(ks**2)
    features = [
        pd.concat([feat_df, pd.DataFrame(np.array(
            [cv2.filter2D(img, -1, individual[(count + i*k_len):(count + (i + 1) * k_len)].reshape((ks, ks)))[mask]
             for i in range(n_kernels // len(k_size))]
        ).T, columns=np.arange(j * (n_kernels // len(k_size)), (j + 1) * (n_kernels // len(k_size))))], axis=1)
        for img, mask, feat_df in zip(IMAGES, MASKS, features)
    ]
    count += k_len * (n_kernels // len(k_size))
    
conv_features = pd.concat(features, ignore_index=True)

conv_features.index = df_train.index

In [14]:
cross_validate_features({
    'x_train': pd.concat([df_train, conv_features], axis=1),
    'y_train': y_train,
})

Accuracy score: 0.9369410086679884

F1 score: 0.7129948106245284

Confusion matrix:

     t/p    0.0   1.0 
      0.0 585122 20187 
      1.0 22786 53378 
Sensivity: 0.7008297883514522
Specificity: 0.9666500911104907
-------------------------------------
Accuracy score: 0.9239774765503249

F1 score: 0.678995561349598

Confusion matrix:

     t/p    0.0   1.0 
      0.0 573783  9022 
      1.0 42687 54688 
Sensivity: 0.561622593068036
Specificity: 0.9845196935510162
-------------------------------------
Accuracy score: 0.9408904818143721

F1 score: 0.752745905658527

Confusion matrix:

     t/p    0.0   1.0 
      0.0 580716 12728 
      1.0 27612 61406 
Sensivity: 0.689815542923903
Specificity: 0.9785523149614791
-------------------------------------
Accuracy score: 0.9419756318539088

F1 score: 0.6993361938222583

Confusion matrix:

     t/p    0.0   1.0 
      0.0 594572 12602 
      1.0 26849 45881 
Sensivity: 0.6308400934964938
Specificity: 0.9792448293240488
----------------------

{'acc': 0.9298430340680646, 'f1': 0.698874485662644}

In [15]:
parameters = [22,  7, 66, 75, 39, 72, 35, 35]
PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/images'
MASK_PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/mask'
LABELS_PATH = r'/home/fer/Drive/Estudios/Master-IA/TFM/dataset/DRIVE/training/1st_manual'


def load_images():
    paths = [f"{PATH}/{path}" for path in sorted(listdir(PATH))]
    return [np.asarray(Image.open(path).convert('RGB'))[:, :, 1] for path in paths]

def load_masks():
    paths = [f"{MASK_PATH}/{path}" for path in sorted(listdir(MASK_PATH))]
    return [np.asarray(Image.open(path).convert('L')) > 100 for path in paths]

P_OBJ = Preprocess(
    lbp_radius=1,
    lbp_method=PARAMETERS.LBP_METHOD,
    height={'DRIVE': 608, 'CHASE': 960, 'STARE': 608}[PARAMETERS.DATASET],
    width={'DRIVE': 576, 'CHASE': 1024, 'STARE': 704}[PARAMETERS.DATASET],
    balance=PARAMETERS.BALANCE
)

IMAGES = [Preprocess.img_processing(img, params=parameters) for img in load_images()]
MASKS = load_masks()

img_df = [pd.DataFrame(img[mask]) for img, mask in zip(IMAGES, MASKS)]

gray_intensity_train = pd.concat(img_df[:14], ignore_index=True)
gray_intensity_train.columns = ['gray_optimized']
gray_intensity_train.index = df_train.index

In [16]:
cross_validate_features({
    'x_train': pd.concat([df_train, conv_features, gray_intensity_train], axis=1),
    'y_train': y_train,
})

Accuracy score: 0.9365638844092136

F1 score: 0.7144876231738567

Confusion matrix:

     t/p    0.0   1.0 
      0.0 584152 21157 
      1.0 22073 54091 
Sensivity: 0.7101911664303345
Specificity: 0.965047603785835
-------------------------------------
Accuracy score: 0.9250007350995325

F1 score: 0.6826977502161461

Confusion matrix:

     t/p    0.0   1.0 
      0.0 574288  8517 
      1.0 42496 54879 
Sensivity: 0.563584082156611
Specificity: 0.9853861926373315
-------------------------------------
Accuracy score: 0.93930504555565

F1 score: 0.7478757334501984

Confusion matrix:

     t/p    0.0   1.0 
      0.0 579605 13839 
      1.0 27583 61435 
Sensivity: 0.6901413197330877
Specificity: 0.9766801922338081
-------------------------------------
Accuracy score: 0.942884583705935

F1 score: 0.7060088273815383

Confusion matrix:

     t/p    0.0   1.0 
      0.0 594443 12731 
      1.0 26102 46628 
Sensivity: 0.6411109583390623
Specificity: 0.9790323696337458
-----------------------

{'acc': 0.93026445369877, 'f1': 0.7014782078988748}

In [1]:
features = {
    'x_train': pd.concat([df_train, conv_features], axis=1),
    'x_test': pd.concat([df_test, conv_features_test], axis=1),
    'y_train': y_train,
    'y_test': y_test
}

from main import main
PARAMETERS.PLOT = True

main(lgb='Num', features=features, plot_once=True, recurrence=True)

NameError: name 'pd' is not defined